In [1]:
##### This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
df  = pd.read_csv('/kaggle/input/last-convert/submission(1).csv')
df = df.drop('Unnamed: 0',axis=1)

In [3]:
df['isna'] = df['text_conv'].isna()

In [4]:
df['isna'].sum()

584

In [5]:
df = df[~df['isna']]

In [6]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("feature-extraction", model="nielsr/lilt-xlm-roberta-base")

2024-06-22 13:05:46.904667: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-22 13:05:46.904789: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-22 13:05:47.052011: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


config.json:   0%|          | 0.00/774 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.14G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/451 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

In [7]:
def get_emb(text):
    try:
        return pipe(text)[0][-1]
    except:
        return None

In [8]:
import pandas as pd
import requests

from tqdm import tqdm
tqdm.pandas()

# from pandarallel import pandarallel
# pandarallel.initialize(progress_bar=True)

In [9]:
df['emv'] = df['text_conv'].progress_apply(get_emb)

100%|██████████| 2250/2250 [04:47<00:00,  7.82it/s]


In [10]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)


In [11]:
#  Params: 
#     svc_c: 9.352476690815911e-05
#     svc_kernel: linear
#     svc_gamma: 0.6056404647479878


In [12]:
import optuna
import numpy as np
import pandas as pd
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score
from catboost import CatBoostClassifier


import optuna
import numpy as np
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.datasets import make_classification

In [13]:
df = df[~df['emv'].isna()]

In [14]:
data = np.array(df['emv'].tolist())

In [15]:
data.shape

(2250, 768)

In [16]:
df['Успешный результат'].unique()

array([ True, False])

In [17]:
pipeline = Pipeline([
        ('scaler', StandardScaler()),
        ('svc', SVC(C=0.015564950023394423, kernel='rbf', gamma=0.02719557109129227,probability=True))
    ])
# #    {'svc_c': 0.01692649830942587, 'svc_kernel': 'rbf', 'svc_gamma': 0.02381086536079604}
# svc_c: 0.015564950023394423
#     svc_kernel: rbf
#     svc_gamma: 0.02719557109129227
pipeline.fit(data, df['Успешный результат'])
# Create an Optuna study


Pipeline(steps=[('scaler', StandardScaler()),
                ('svc',
                 SVC(C=0.015564950023394423, gamma=0.02719557109129227,
                     probability=True))])

In [18]:
# test[:,0]

In [19]:
df['proba'] = pipeline.predict_proba(data)[:,0]

In [20]:
df['proba']

0       0.664448
1       0.683077
2       0.683077
3       0.683077
4       0.657093
          ...   
2826    0.129748
2828    0.238654
2829    0.701842
2830    0.656703
2831    0.748490
Name: proba, Length: 2250, dtype: float64

In [21]:
df = df.drop(['text_conv','isna','emv'],axis=1)

In [22]:
df.isna().sum()

Тип                          0
Статус                       0
Время                        0
Схема                       13
Откуда                       0
Куда                         0
Кто ответил                  0
Длительность звонка          0
Длительность разговора       8
Время ответа                 0
Оценка                    2250
ID записи                    0
Метка                     2250
Теги                      2250
ID заказа звонка          2246
Запись существует            0
Новый клиент                 0
Состояние перезвона       2250
Время перезвона           2250
Успешный результат           0
Ответственный из CRM         4
proba                        0
dtype: int64

In [23]:
df = df.drop(['Время перезвона','Состояние перезвона','ID заказа звонка','Теги','Метка','Оценка'],axis=1)

In [24]:
df.isna().sum()

Тип                        0
Статус                     0
Время                      0
Схема                     13
Откуда                     0
Куда                       0
Кто ответил                0
Длительность звонка        0
Длительность разговора     8
Время ответа               0
ID записи                  0
Запись существует          0
Новый клиент               0
Успешный результат         0
Ответственный из CRM       4
proba                      0
dtype: int64

In [25]:
df

,Тип,Статус,Время,Схема,Откуда,Куда,Кто ответил,Длительность звонка,Длительность разговора,Время ответа,ID записи,Запись существует,Новый клиент,Успешный результат,Ответственный из CRM,proba
0,Входящий,Отвечен,18.05.2022 11:33:47,Евген,7965,+7962,205,28,12.0,16,1652862827,1,1,True,205 Евгения Фатун,0.664448
1,Входящий,Отвечен,18.05.2022 11:34:56,Федор,7965,+7903,219,31,12.0,19,1652862896,1,0,False,219 Софья Султанова,0.683077
2,Входящий,Отвечен,19.05.2022 18:02:22,Татья,7928,+7903,209,56,27.0,29,1652972541,1,1,True,209 Татьяна Полубоярова,0.683077
3,Входящий,Отвечен,19.05.2022 18:13:45,Татья,7938,+7903,209,160,148.0,12,1652973225,1,0,False,209 Татьяна Полубоярова,0.683077
4,Входящий,Отвечен,19.05.2022 20:34:34,Татья,7905,+7903,209,173,157.0,16,1652981674,1,1,True,209 Татьяна Полубоярова,0.657093
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2826,Входящий,Отвечен,26.07.2022 15:21:21,8800,7904,envyz,210,41,34.0,7,1658838081,1,1,True,210 Ирина Подшивалова,0.129748
2828,Входящий,Отвечен,26.07.2022 15:24:14,8800,7891,envyz,210,30,24.0,6,1658838254,1,1,True,210 Ирина Подшивалова,0.238654
2829,Исходящий,Отвечен,26.07.2022 15:29:00,Бойцо,202,+7918,+7918,119,102.0,17,1658838540,1,0,False,202 Анна Бойцова,0.701842
2830,Входящий,Отвечен,26.07.2022 16:05:15,8800,7967,88003,210,44,37.0,7,1658840715,1,1,True,221 Анна Суворова,0.656703


In [26]:
for i in df.columns:
    print(i,len(df[i].value_counts()))

Тип 2
Статус 1
Время 2246
Схема 20
Откуда 117
Куда 77
Кто ответил 90
Длительность звонка 559
Длительность разговора 552
Время ответа 62
ID записи 2246
Запись существует 1
Новый клиент 2
Успешный результат 2
Ответственный из CRM 22
proba 1511


In [27]:
df = df.drop(['ID записи'],axis=1)

In [28]:
df['Время'].dtype

dtype('O')

In [29]:
def conv_time(ss):
    return pd.to_datetime(ss, format='%d.%m.%Y %H:%M:%S')


In [30]:
df['Время'] = df['Время'].progress_apply(conv_time)

100%|██████████| 2250/2250 [00:00<00:00, 6431.45it/s]


In [31]:
df

,Тип,Статус,Время,Схема,Откуда,Куда,Кто ответил,Длительность звонка,Длительность разговора,Время ответа,Запись существует,Новый клиент,Успешный результат,Ответственный из CRM,proba
0,Входящий,Отвечен,2022-05-18 11:33:47,Евген,7965,+7962,205,28,12.0,16,1,1,True,205 Евгения Фатун,0.664448
1,Входящий,Отвечен,2022-05-18 11:34:56,Федор,7965,+7903,219,31,12.0,19,1,0,False,219 Софья Султанова,0.683077
2,Входящий,Отвечен,2022-05-19 18:02:22,Татья,7928,+7903,209,56,27.0,29,1,1,True,209 Татьяна Полубоярова,0.683077
3,Входящий,Отвечен,2022-05-19 18:13:45,Татья,7938,+7903,209,160,148.0,12,1,0,False,209 Татьяна Полубоярова,0.683077
4,Входящий,Отвечен,2022-05-19 20:34:34,Татья,7905,+7903,209,173,157.0,16,1,1,True,209 Татьяна Полубоярова,0.657093
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2826,Входящий,Отвечен,2022-07-26 15:21:21,8800,7904,envyz,210,41,34.0,7,1,1,True,210 Ирина Подшивалова,0.129748
2828,Входящий,Отвечен,2022-07-26 15:24:14,8800,7891,envyz,210,30,24.0,6,1,1,True,210 Ирина Подшивалова,0.238654
2829,Исходящий,Отвечен,2022-07-26 15:29:00,Бойцо,202,+7918,+7918,119,102.0,17,1,0,False,202 Анна Бойцова,0.701842
2830,Входящий,Отвечен,2022-07-26 16:05:15,8800,7967,88003,210,44,37.0,7,1,1,True,221 Анна Суворова,0.656703


In [32]:
df.isna().sum()

Тип                        0
Статус                     0
Время                      0
Схема                     13
Откуда                     0
Куда                       0
Кто ответил                0
Длительность звонка        0
Длительность разговора     8
Время ответа               0
Запись существует          0
Новый клиент               0
Успешный результат         0
Ответственный из CRM       4
proba                      0
dtype: int64

In [33]:
df.dtypes

Тип                               object
Статус                            object
Время                     datetime64[ns]
Схема                             object
Откуда                             int64
Куда                              object
Кто ответил                       object
Длительность звонка                int64
Длительность разговора           float64
Время ответа                       int64
Запись существует                  int64
Новый клиент                       int64
Успешный результат                  bool
Ответственный из CRM              object
proba                            float64
dtype: object

In [34]:
df.select_dtypes(exclude=['bool','float64','int64','datetime64[ns]']).columns


Index(['Тип', 'Статус', 'Схема', 'Куда', 'Кто ответил',
       'Ответственный из CRM'],
      dtype='object')

In [35]:
X = df.drop('Успешный результат',axis=1)
y = df['Успешный результат']

In [36]:
X.columns

Index(['Тип', 'Статус', 'Время', 'Схема', 'Откуда', 'Куда', 'Кто ответил',
       'Длительность звонка', 'Длительность разговора', 'Время ответа',
       'Запись существует', 'Новый клиент', 'Ответственный из CRM', 'proba'],
      dtype='object')

In [37]:
from catboost import Pool

In [38]:
X['Схема'] = X['Схема'].astype(str)
X['Ответственный из CRM'] = X['Ответственный из CRM'].astype(str)



In [39]:
pool_all = Pool(
    X,
    label=y,
    cat_features=['Тип', 'Статус', 'Схема', 'Куда', 'Кто ответил','Ответственный из CRM'],
    feature_names=['Тип', 'Статус', 'Время', 'Схема', 'Откуда', 'Куда', 'Кто ответил',
       'Длительность звонка', 'Длительность разговора', 'Время ответа',
       'Запись существует', 'Новый клиент', 'Ответственный из CRM', 'proba'])

In [40]:
from catboost import cv

In [53]:
 params = dict(    iterations=20,
                         learning_rate=1e-3,
                         depth= 10,
        eval_metric='Precision',
                   loss_function='CrossEntropy',
        verbose= False
                 )
cv(pool=pool_all,
   params=params,
   fold_count=5,
        verbose= False,

   type='Classical',
  )#['test-Precision-mean'].mean()

Training on fold [0/5]

bestTest = 0.9383886256
bestIteration = 7

Training on fold [1/5]

bestTest = 0.9191489362
bestIteration = 10

Training on fold [2/5]

bestTest = 0.92
bestIteration = 7

Training on fold [3/5]

bestTest = 0.9471153846
bestIteration = 5

Training on fold [4/5]

bestTest = 0.9429824561
bestIteration = 6



,iterations,test-Precision-mean,test-Precision-std,train-Precision-mean,train-Precision-std,test-CrossEntropy-mean,test-CrossEntropy-std,train-CrossEntropy-mean,train-CrossEntropy-std
0,0,0.891190,0.013820,0.901377,0.009139,0.691524,0.000112,0.691503,0.000052
1,1,0.912890,0.007121,0.917370,0.003298,0.689843,0.000128,0.689784,0.000048
2,2,0.917894,0.013753,0.922548,0.003441,0.688117,0.000178,0.688057,0.000115
3,3,0.914080,0.007721,0.922131,0.003894,0.686403,0.000230,0.686332,0.000098
4,4,0.922605,0.011869,0.926090,0.003148,0.684706,0.000270,0.684623,0.000136
5,5,0.928969,0.016244,0.929693,0.003302,0.683022,0.000330,0.682936,0.000147
6,6,0.928897,0.013385,0.931339,0.004855,0.681507,0.000327,0.681392,0.000212
7,7,0.928632,0.013992,0.932319,0.005440,0.679873,0.000366,0.679738,0.000263
8,8,0.928650,0.011572,0.931234,0.004268,0.678225,0.000420,0.678067,0.000247
9,9,0.925518,0.012005,0.931591,0.005003,0.676562,0.000402,0.676357,0.000312


In [ ]:
def catboost_objective(trial):
  
    params = {
        "objective": trial.suggest_categorical("objective", ["Logloss", "CrossEntropy"]),
        "colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.01, 0.1),
        "depth": trial.suggest_int("depth", 1, 12),
        "boosting_type": trial.suggest_categorical("boosting_type", ["Ordered", "Plain"]),
        "bootstrap_type": trial.suggest_categorical(
            "bootstrap_type", ["Bayesian", "Bernoulli", "MVS"]
        ),
        "verbose":False,
        "eval_metric":'Precision',

    }
    if params["bootstrap_type"] == "Bayesian":
        params["bagging_temperature"] = trial.suggest_float("bagging_temperature", 0, 10)
    elif params["bootstrap_type"] == "Bernoulli":
        params["subsample"] = trial.suggest_float("subsample", 0.1, 1)
      

    scores=cv(pool=pool_all,
       params=params,
       fold_count=5,

       type='Classical',
       )['test-Precision-mean'].mean()
    
  
    return scores

# Create an Optuna study for the CatBoost Classifier
cb_study = optuna.create_study(direction='maximize')
cb_study.optimize(catboost_objective, n_trials=200)

# Get the best parameters for the CatBoost Classifier


In [55]:
print("Best trial:")
trial = cb_study.best_trial
print(f"  Value: {trial.value}")
print("  Params: ")
for key, value in trial.params.items():
    print(f"    {key}: {value}")



Best trial:
  Value: 0.9432857110736063
  Params: 
    objective: Logloss
    colsample_bylevel: 0.09440173601958003
    depth: 12
    boosting_type: Ordered
    bootstrap_type: Bernoulli
    subsample: 0.9584094346366171
